## Agents
### Your Personal Health Expert 

- Agents are systems that use an LLM as a reasoning engine to determine which actions to take and what the inputs to those actions should be.
- The results of those actions can then be fed back into the agent and it determines whether more actions are needed, or whether it is okay to finish.

In [1]:
from dotenv import load_dotenv
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool



llm = ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-31T17:38:24.1435259Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 412465600, 'load_duration': 16415300, 'prompt_eval_count': 26, 'prompt_eval_duration': 243044000, 'eval_count': 10, 'eval_duration': 150506000}, id='run-c06ecfa6-30b6-4449-9b1a-af0bdc4288c7-0', usage_metadata={'input_tokens': 26, 'output_tokens': 10, 'total_tokens': 36})

### Define Tools
#### 1. Tavily

In [3]:
from langchain_community.tools import TavilySearchResults

@tool
def search(query: str) -> str:
    """Search Tavily for current and latest news and updates.
        For example, stock market news, weather updates, news etc.
    Args:
        query: The search query.
    """
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True
    )
    response = search.invoke(query)
    return response

In [4]:
# tavily_search("stock market news")
search.invoke("stock market news")

[{'url': 'https://www.cnbc.com/stocks/',
  'content': 'Get the latest stock market news, analysis and data from CNBC.com. Find out which stocks are rising or falling, what analysts are saying, and how to invest in the current market environment.'},
 {'url': 'https://finance.yahoo.com/news/live/stock-market-today-nasdaq-sp-500-sink-as-meta-microsoft-revive-big-techs-ai-spending-worries-133028993.html',
  'content': "Stock market today: Nasdaq, S&P 500 sink as Meta, Microsoft revive Big Tech's AI spending worries Stock market today: Nasdaq, S&P 500 sink as Meta, Microsoft revive Big Tech's AI spending worries The Nasdaq led a tumble in US stocks on Thursday after Meta (META) and Microsoft (MSFT) earnings sparked worries about prospects for Big Techs amid rising artificial intelligence costs. Meta raised the lower end of its guidance range for full-year spending from $37 billion to $38 billion, and CFO Susan Li said the company expects “significant capital expenditure growth in 2025.” Mic

#### 2. Health Supplements Retriever

In [5]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

db_name = r"D:\NLP\LLM\Langchain and Ollama\09. Vector Stores and Retrievals\health_supplements"
vector_store = FAISS.load_local(db_name, embeddings, allow_dangerous_deserialization=True)

retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

In [6]:
@tool
def health_supplements(query: str) -> str:
    """Search for information about Health Supplements.
    For any questions about Health and Gym Supplements, you must use this tool!,

    Args:
        query: The search query.
    """
    response = retriever.invoke(query)
    return response

In [7]:
question = "What is the best supplement for muscle gain?"
health_supplements.invoke(question)

[Document(metadata={'source': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 0, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and high-\nintensity activities, which is the choice of consumers [4].\nCreatine monohydrate is another well-known supplement used to gain muscle mass\nand support performance and recovery. It is known not to increase fat mass an

### Agent Creation

 - The agent is responsible for taking in input and deciding what actions to take
 - the Agent does not execute those actions - that is done by the AgentExecutor
 - `create_tool_calling_agent` will call `.bind_tools` for us under the hood.

In [8]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
tools = [search, health_supplements]
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [12]:
question = "What is the best supplement for muscle gain?"
# question = "What's the weather in SF?"

response = agent_executor.invoke({'input': question})
print(response['input'])
print("\n\nOutput:")
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `health_supplements` with `{'query': 'best supplements for muscle gain'}`


[Document(metadata={'source': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'page': 1, 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113850Z', 'trapped': ''}, page_content='Foods 2024, 13, 1424\n2 of 21\nand sports industry, evidence suggests that creatine can benefit not only athletes but also\nthe elderly and the general population [6]. Branched-chain amino acids (BCAA) also offer\na plethora of benefits for consumers. As explained by Sanz et al. [7], BCAAs are stored\ndirectly in muscles and serve as the raw materials needed to build new muscle. This\ncontributes to the overall process of strengthening musc

In [13]:
question = "What's the weather in SF?"

response = agent_executor.invoke({'input': question})
print(response['input'])
print("\n\nOutput:")
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'SF weather'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1730398350, 'localtime': '2024-10-31 11:12'}, 'current': {'last_updated_epoch': 1730397600, 'last_updated': '2024-10-31 11:00', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 195, 'wind_dir': 'SSW', 'pressure_mb': 1021.0, 'pressure_in': 30.16, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 69, 'cloud': 50, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 13.5, 'windchill_f': 56.4, 'heatindex_c': 13.8, 'heatindex_f': 56.9, 'dewpoint_c': 10.8, 'dewpoint_f': 51.5, 'vis_km': 16.0, 'vis_miles': 9.0,

In [14]:
question = "Write Report on the stock market news"

response = agent_executor.invoke({'input': question})
print(response['input'])
print("\n\nOutput:")
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'stock market news'}`


[{'url': 'https://www.cnn.com/business/live-news/stock-market-news-today/index.html/', 'content': 'Stock market today: Latest news | CNN Business Follow CNN Business CNN10 About CNN Markets: The Dow and S&P 500 closed higher for the third straight day, notching their best streak since November. Sears avoids liquidation: The retailer announced an 11th-hour deal to stay in business, at least for now. Gold always performs well when investors are nervous, as they clearly are right now, says CNN Business’ Paul R. CNN10 About CNN Follow CNN Business Dow Jones: The Dow Jones branded indices are proprietary to and are calculated, distributed and marketed by DJI Opco, a subsidiary of S&P Dow Jones Indices LLC and have been licensed for use to S&P Opco, LLC and CNN.'}, {'url': 'https://www.cnbc.com/stocks/', 'content': 'Get the latest stock market news, analysis and data from CNBC.com. Find out wh